Notebook dealing with the final preprocessing steps for the neural networks.


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.utils.data
import pandas as pd
from torch.utils import data
from numpy import array
from numpy import argmax
import argparse
from torch.autograd import Variable
from torch import optim
import numpy as np
import os
from sklearn.model_selection import train_test_split
import logging
import pickle as pkl
import warnings
pd.options.mode.chained_assignment = None
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"
import glob
import random
random.seed(1991)
#torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [3]:
torch.manual_seed(1991)
if not torch.cuda.is_available():
    
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = torch.device("cuda")

In [7]:
heads=pd.read_pickle("/data/dharp/compounding/datasets/heads_CompoundCentric_DecadeCentric_300.pkl")
heads.reset_index(inplace=True)
heads=heads.drop(['decade'],axis=1).groupby(['head']).mean()
heads.info()
heads.head()

<class 'pandas.core.frame.DataFrame'>
Index: 53466 entries, a_n to zygote_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 122.8+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
head,,,,,,,,,,,,,,,,,,,,,
a_n,0.536217,-0.029517,-0.012247,-0.020589,-0.024528,0.199687,0.002459,-0.028174,0.000748,-0.162900,...,-0.058662,0.042939,0.032226,0.045636,0.000804,0.061979,0.025265,0.032431,0.010333,-0.068106
aa_n,0.016471,-0.002655,0.003908,0.000666,-0.001233,0.027962,0.000181,-0.006771,0.005269,0.002151,...,-0.005486,0.081462,0.044021,0.055576,-0.031826,0.043881,0.017116,-0.012633,-0.003582,-0.070446
aaa_n,0.369783,-0.008786,-0.033639,-0.011358,-0.021063,0.152685,-0.001936,-0.020756,0.049742,-0.122735,...,-0.036469,0.041279,0.011177,0.026513,-0.033482,0.040557,0.026827,0.034848,-0.025072,-0.068561
aaaaa_n,0.001559,-0.000158,0.000625,0.000832,-0.000796,0.004964,0.000146,-0.000465,0.002441,-0.007268,...,-0.063368,0.120493,0.037918,0.045026,-0.030933,0.076486,0.060778,0.095964,-0.030948,-0.158763
aaai_n,0.000988,0.000103,0.001039,0.002720,-0.000062,0.001140,0.010847,0.000047,0.031081,-0.002763,...,0.005658,-0.004508,-0.002646,-0.020482,0.093074,0.036829,0.048474,-0.106865,-0.014324,-0.063920


In [8]:
modifiers=pd.read_pickle("/data/dharp/compounding/datasets/modifiers_CompoundCentric_DecadeCentric_300.pkl")
modifiers.reset_index(inplace=True)
modifiers=modifiers.drop(['decade'],axis=1).groupby(['modifier']).mean()
modifiers.info()
modifiers.head()

<class 'pandas.core.frame.DataFrame'>
Index: 54400 entries, a_n to zylobalsamum_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 124.9+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
modifier,,,,,,,,,,,,,,,,,,,,,
a_n,0.250488,-0.016303,0.002421,0.044242,-0.030004,0.080336,-0.005189,-0.011844,0.024295,-0.076888,...,-0.009632,0.081663,0.058108,0.024490,-0.053609,-0.097451,0.093379,-0.004846,-0.065470,-0.238026
aa_n,0.068528,-0.004179,0.002100,-0.001845,-0.002256,0.015144,0.007321,-0.004580,0.004887,-0.023613,...,-0.059578,0.108445,0.028685,0.041453,-0.016382,0.060577,0.060166,0.083766,-0.017441,-0.146023
aaa_n,0.021914,-0.002406,0.011616,0.023197,-0.009332,0.002951,0.000239,-0.001105,0.002711,-0.008170,...,-0.058486,0.114269,0.047182,0.046012,-0.032943,0.074439,0.064875,0.087735,-0.028544,-0.159535
aaaa_n,0.001559,-0.000158,0.000625,0.000832,-0.000796,0.004964,0.000146,-0.000465,0.002441,-0.007268,...,-0.063368,0.120493,0.037918,0.045026,-0.030933,0.076486,0.060778,0.095964,-0.030948,-0.158763
aaaaa_n,0.001559,-0.000158,0.000625,0.000832,-0.000796,0.004964,0.000146,-0.000465,0.002441,-0.007268,...,-0.063368,0.120493,0.037918,0.045026,-0.030933,0.076486,0.060778,0.095964,-0.030948,-0.158763


In [9]:
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
m, h = zip(*novel_compounds_list)
heads_list=list(set(h))
modifiers_list=list(set(m))

In [10]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds.columns=['modifier','head']

In [11]:
positive_heads=pd.merge(novel_compounds.drop('modifier',axis=1),heads.reset_index(),on=["head"])

positive_modifiers=pd.merge(novel_compounds.drop('head',axis=1),modifiers.reset_index(),on=["modifier"])
#positive_df['Plausibility']=True

In [12]:
positive_heads.head()

,head,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,peaceful_n,0.084673,0.004526,0.350015,-0.035577,0.021538,-0.047361,0.083481,-0.013165,0.014318,...,-0.008103,0.002998,0.009874,0.001091,0.006758,0.000884,-0.007504,-0.003493,0.002939,0.003710
1,peaceful_n,0.084673,0.004526,0.350015,-0.035577,0.021538,-0.047361,0.083481,-0.013165,0.014318,...,-0.008103,0.002998,0.009874,0.001091,0.006758,0.000884,-0.007504,-0.003493,0.002939,0.003710
2,robinson_n,0.690679,-0.055788,-0.069111,0.024073,-0.038583,0.038354,0.002031,-0.009363,-0.038313,...,0.037275,-0.041498,0.001673,0.018245,0.020412,0.019159,0.008809,-0.078762,-0.028517,-0.029286
3,robinson_n,0.690679,-0.055788,-0.069111,0.024073,-0.038583,0.038354,0.002031,-0.009363,-0.038313,...,0.037275,-0.041498,0.001673,0.018245,0.020412,0.019159,0.008809,-0.078762,-0.028517,-0.029286
4,supply_n,0.774867,-0.055282,-0.030543,-0.022465,-0.017167,0.228280,0.007839,-0.026089,0.005531,...,-0.007130,0.042597,-0.007180,-0.007903,0.018143,0.023752,0.012634,-0.000541,-0.006356,0.012804


In [13]:
positive_heads_tensor = torch.tensor(positive_heads.drop('head',axis=1).values)
positive_heads_tensor.shape

torch.Size([28448, 300])

In [14]:
positive_modifiers_tensor = torch.tensor(positive_modifiers.drop('modifier',axis=1).values)
positive_modifiers_tensor.shape

torch.Size([28448, 300])

In [15]:
positive_Y=torch.ones(positive_modifiers_tensor.shape[0])
positive_Y.shape

torch.Size([28448])

In [16]:
positive_class=torch.cat((positive_modifiers_tensor, positive_heads_tensor), 1)
positive_class.shape

torch.Size([28448, 600])

In [17]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    
    df.columns=['modifier','head']
    negative_heads=pd.merge(df.drop('modifier',axis=1),heads.reset_index(),on=["head"])
    negative_modifiers=pd.merge(df.drop('head',axis=1),modifiers.reset_index(),on=["modifier"])
    negative_heads_tensor = torch.tensor(negative_heads.drop('head',axis=1).values)
    negative_modifiers_tensor = torch.tensor(negative_modifiers.drop('modifier',axis=1).values)

    negative_Y=torch.zeros(negative_modifiers_tensor.shape[0])
    negative_class=torch.cat((negative_modifiers_tensor, negative_heads_tensor), 1)

    return negative_class,negative_Y

In [18]:
def tensor_joiner(files):
    tensor_list=[]
    for file in files:
        negative_class,negative_Y=neg_df_creator(file)
        X=torch.cat((positive_class, negative_class), 0)
        Y=torch.cat((positive_Y,negative_Y),0)
        tensor_list.append([X,Y])
    
    return tensor_list

In [19]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=tensor_joiner(corrupt_modifier_files)

In [20]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=tensor_joiner(corrupt_head_files)

In [21]:
input_size = 600
hidden_size = 300
num_classes = 2
num_epochs = 50
batch_size = 100
learning_rate = 0.001

In [22]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes,bias=False)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [23]:
def looper(datasets):
    total_accuracy=[]
    
    for dataset in datasets:
        X=dataset[0]
        Y=dataset[1]
        n = len(X)  # how many total elements you have
        n_test = int( n * .2 )  # number of test/val elements
        n_train = n - n_test
        idx = list(range(n))  # indices to all elements
        random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
        train_idx = idx[:n_train]
        test_idx = idx[n_train:]
        trX=X[train_idx].float().to(device)
        teX=X[test_idx].float().to(device)

        trY=Y[train_idx].long().to(device)
        teY=Y[test_idx].long().to(device)
        model = NeuralNet(input_size, hidden_size, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        n_examples=trX.shape[0]
        for i in range(num_epochs):

            cost = 0.

            num_batches = n_examples // batch_size
            for k in range(num_batches):
                start, end = k * batch_size, (k + 1) * batch_size
                outputs = model(trX[start:end])
                loss = criterion(outputs, trY[start:end])

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            #if (k+1) % 100 == 0:
            #print ('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, num_epochs, loss.item()))

        with torch.no_grad():
            correct=0
            total=0
            outputs = model(teX)
            _, predicted = torch.max(outputs.data, 1)
            total += teY.size(0)
            correct += (predicted == teY).sum().item()
        curr_acc=100 * correct / total
        print(curr_acc)
        total_accuracy.append(curr_acc)
    return total_accuracy
        

In [24]:
cor_head_acc=looper(corrupt_heads)

83.2059056156077
82.82801652166272
82.704982863169
82.14254328148344
83.13560066789701
82.47649178310924
82.27436505844099
82.50285613850075
82.90710958783724
82.67861850777749


82.91589770630108
83.12681254943317
82.59952544160295
82.76649969241585
83.14438878636084
83.05650760172247
82.90710958783724
83.22348185253537
82.67861850777749
82.74892345548818

In [25]:
round(np.mean(cor_head_acc),2)

82.69

In [26]:
round(np.std(cor_head_acc),2)

0.33

In [27]:
cor_mod_acc=looper(corrupt_modifiers)

82.50285613850075
82.6610422708498
82.50285613850075
81.52737498901485
83.08287195711398
82.27436505844099
83.14438878636084
82.4677036646454
83.06529572018631
82.26557693997715


82.88074523244573
83.40803234027595
82.29194129536866
81.87889972756832
82.72255910009667
82.21284822919412
82.45891554618156
82.5643729677476
82.6610422708498
82.3007294138325

In [28]:
round(np.mean(cor_mod_acc),2)

82.55

In [29]:
round(np.std(cor_mod_acc),2)

0.46